In [2]:
import pandas as pd, gzip, re, 

In [3]:
# Input
input_path = '../Data/schema_Product.gz'
phone_lst = ['smartphone', 'phone',
             'phone case', 'phone cable', 'phone charger', 'phone mount',
             'cell phone']

# Output
output_path = '../Data/phone_sites.txt'

In [ ]:
taxo_pattern = re.compile("<http://schema.org/Product/category>", re.IGNORECASE)
split_pattern = re.compile("^(_:.*)\s(<http:\/\/schema\.org\/Product\/category>)\s(.*)\s<(.*)>\s\.$", re.IGNORECASE)

# Product -> look for product cat -> get source
with gzip.open(input_path,"rt") as f:
    i = 0
    detected = 0
    skipped = 0
    not_taxo = 0
    inserted = 0
    for line in iter(f.readline, ""):
        i += 1
        if not taxo_pattern.search(line):
            not_taxo += 1
            continue
        match = split_pattern.match(line)
        if match is None:
            skipped += 1
            continue
        props = match.groups()
        subj = props[0]
        pred = props[1]
        obj = props[2]
        source = props[3]
        try:
            obj = str(obj).lower()
        except:
            pass
        else:
            pass
        if any(ext in obj for ext in phone_lst):
            detected += 1
            with open(output_path,'a') as file:
                file.write( source + "\n")
                inserted += 1

print("detected " + str(detected) + ", inserted: " + str(inserted) +" lines out of " + str(i) + "; not in taxo: " + str(not_taxo))
print("Done extracting phone & accessories websites from Product.gz file")

In [9]:
df = pd.read_csv(output_path, sep=" ", header=None)
df.columns = ["url"]
df.describe()

,url
count,26821
unique,15823
top,https://www.colamco.com/boss-hp12-headphone-hp...
freq,2296


In [11]:
# Was will ich tun: URL selektieren nach 
#     keine Dupliate
df = df.drop_duplicates(subset='url', keep='first')
df.describe()

,url
count,15823
unique,15823
top,https://i-spare.ru/zapchasti-dlya-iphone/zapch...
freq,1


In [13]:
#     contains phone or smartphone
df[df['url'].str.contains("phone")]
df.head()
#     is from english domain

,url
0,http://www.speckproducts.com/apple/iphone-case...
1,http://www.speckproducts.com/apple/iphone-case...
2,http://www.speckproducts.com/apple/iphone-case...
3,https://www.crutchfield.com/S-wvUmH5Cml0s/p_59...
4,https://www.homechoicestores.com/home-choice-home


In [ ]:
df = pd.read_csv(prodwebsites, sep=" ", header=None)
df.columns = ["url"]
df.describe()

def getNetloc(row):
    try:

        return urlparse(row['url']).netloc
    except:
        print("expection: ", row['url'])
    else:
        print("sad", row['url'])

def getSuffix(row):
    try:
        return tldextract.extract(row['netloc']).suffix
    except:
        print("expection: ", row['url'])
    else:
        print("sad", row['url'])

df['netloc'] = df.apply(getNetloc, axis = 1)
df['suffix'] = df.apply(getSuffix, axis = 1)
df.describe()

In [ ]:
df['netloc'].value_counts().index.tolist()

In [ ]:
# list of target products and synonyms
movie_lst = ['tv series', 'movies', 'film', 'motion picture', 'movies and tv', 'movies & tv']

df_movies = pd.DataFrame(movie_lst, columns=['CATEGORY'])
df_phone = pd.DataFrame(phone_lst, columns=['CATEGORY'])

def count_cat(row):
    try:
        return df['prodcat'].str.contains(row['CATEGORY']).sum()
    except:
        print("exception: ", row['CATEGORY'])
    else:
        print("sad: ", row['CATEGORY'])

df_movies['OCCURENCE'] = df_movies.apply(count_cat, axis =1)
df_phone['OCCURENCE'] = df_phone.apply(count_cat, axis =1)

print(df_movies)
print(df_phone)